# ALS User Meeting 2022
This notebook describes methods to extract key information from microCT image stacks
- Create image montage
- Access slices evenly spaced
- Play slices like a video

Created by Dani Ushizima, CAMERA, LBNL - Aug 1st 2022


In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import filters, morphology, io
from glob import glob


# 1. Read a microct image
- from url
- from NERSC
- from Google drive

## Read from NERSC
- discard this portion if running in Colab

In [ ]:
datapath = "/global/cfs/cdirs/als/users/yourname/yourdata/" #update these values
!ls -lt "$datapath"

In [ ]:
image = io.imread(datapath+'bead_pack.tif')

## Read from Google drive 
- discard this portion if running at NERSC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
datapath = "/content/drive/My Drive/Colab Notebooks/ALS User Meeting 2022 colab/data/"
!ls -lt "$datapath"

## Read a multi-tiff file

In [ ]:
def loadFileNames(path,extension):
  ''' Return filename after using colab files.upload - work for 1 file'''
  fnames = glob(path+extension)
  fnames.sort()
  print(path);
  print(f"Number of files: {len(fnames)}")
  return fnames

In [ ]:
extension = '*tif'
fnames = loadFileNames(datapath+'concrete/',extension)

# 2. How to create an image montage?
- 3x3
- with all your slices

In [ ]:
def montage(path,extension,save=False):
        '''Create montage 3x3 '''
        files = loadFileNames(path, extension)
        nfiles = len(files)
        fig, axes = plt.subplots(3, 3, figsize=(10, 10))
        #Plotting subset of slices evenly spaced
        islices = np.linspace(0,nfiles-1,9,dtype=int)
        islices = islices.astype(int)
        for i, n in enumerate(islices):
            img = io.imread(files[n])
            xy = np.unravel_index(i, (3,3))
            axes[xy].imshow(img,cmap='gray')
            axes[xy].set_title('Slice %i'% n)
            axes[xy].set_axis_off()
            #fig.subplots_adjust( wspace = -0.1,  hspace = 0.1 )    
        if(save):
            fig.savefig(path+'MyMontage.png')
            print('Saved@ '+path+'MyMontage.png')

In [ ]:
montage(datapath+'concrete/',extension)

In [ ]:
def readAndShowMosaic(path,extension):
  '''Read and show ALL images from a folder
     Warning: if nfiles too large, select subset of files
  '''
  files = loadFileNames(path, extension)
  nfiles = len(files)
  y = np.ceil(np.sqrt(nfiles)).astype('int')
  x = np.ceil(nfiles/y).astype('int')
  f, ax = plt.subplots(x, y, figsize=(20, 20))
  
  for i,axi in enumerate(ax.flatten()[0:nfiles]):
    img = io.imread(files[i])
    fileCore = str.split(files[i],'/')[-1]
    fileCore = str.split(fileCore,'.')[0]
    axi.imshow(img[::10,::10],cmap='inferno')
    axi.set_axis_off()
    axi.set_title(fileCore)

In [ ]:
readAndShowMosaic(datapath+'concrete/',extension)

## Play your stack

In [ ]:
!pip install --upgrade plotly

In [ ]:
ic = io.ImageCollection(fnames,conserve_memory=True)
fullstack = ic[0:10].concatenate() #warning: this step might require a lot of mem with your data

In [ ]:
import plotly.express as px
downsample = 2 
fig = px.imshow( fullstack[:,::downsample,::downsample] , animation_frame=0, binary_string=True, labels=dict(animation_frame="microct slice"))
fig.show()